In [ ]:
!pip install rdflib
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from rdflib import Graph, Namespace, Literal, RDF, URIRef
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the dataset from CSV
df = pd.read_excel('/content/Medicine_description.xlsx')
# Fill NaN values with an empty string or any suitable placeholder
df = df.fillna('')

In [ ]:
# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Drug_Name'] + ' ' + df['Reason'] + ' ' + df['Description'])
total_words = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(df['Drug_Name'] + ' ' + df['Reason'] + ' ' + df['Description'])
max_sequence_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='post')

In [ ]:
# Convert string labels to integers using LabelEncoder
label_encoder = LabelEncoder()
df['Drug_Name'] = label_encoder.fit_transform(df['Drug_Name'])

# Create the ontology
onto = Graph()
onto_namespace = Namespace("http://example.org/")

# Define ontology classes and properties
onto.add((onto_namespace.Medicine, RDF.type, URIRef("owl:Class")))
onto.add((onto_namespace.hasName, RDF.type, URIRef("owl:ObjectProperty")))
onto.add((onto_namespace.hasSymptoms, RDF.type, URIRef("owl:ObjectProperty")))
onto.add((onto_namespace.hasCauses, RDF.type, URIRef("owl:ObjectProperty")))
onto.add((onto_namespace.hasDisease, RDF.type, URIRef("owl:ObjectProperty")))

<Graph identifier=Nacb463e3548c402cae8bdc8864e26de0 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Populate ontology with data
for index, row in df.iterrows():
    medicine_uri = onto_namespace[row['Drug_Name']]
    onto.add((medicine_uri, RDF.type, onto_namespace.Medicine))
    onto.add((medicine_uri, onto_namespace.hasName, Literal(row['Drug_Name'])))
    onto.add((medicine_uri, onto_namespace.hasSymptoms, Literal(row['Drug_Name'])))
    onto.add((medicine_uri, onto_namespace.hasCauses, Literal(row['Reason'])))
    onto.add((medicine_uri, onto_namespace.hasDisease, Literal(row['Description'])))

In [ ]:
# Define the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(input_sequences, df['Drug_Name'], epochs=10)

Epoch 1/10
703/703 [==============================] - 86s 123ms/step - loss: 10.2498 - accuracy: 4.4482e-05
Epoch 2/10
703/703 [==============================] - 84s 120ms/step - loss: 10.0587 - accuracy: 2.2241e-04
Epoch 3/10
703/703 [==============================] - 90s 128ms/step - loss: 9.1120 - accuracy: 0.0031
Epoch 4/10
703/703 [==============================] - 86s 122ms/step - loss: 6.5268 - accuracy: 0.0502
Epoch 5/10
703/703 [==============================] - 84s 119ms/step - loss: 3.5572 - accuracy: 0.2772
Epoch 6/10
703/703 [==============================] - 84s 119ms/step - loss: 1.6367 - accuracy: 0.6065
Epoch 7/10
703/703 [==============================] - 85s 121ms/step - loss: 0.7000 - accuracy: 0.8290
Epoch 8/10
703/703 [==============================] - 87s 123ms/step - loss: 0.2868 - accuracy: 0.9373
Epoch 9/10
703/703 [==============================] - 85s 120ms/step - loss: 0.1168 - accuracy: 0.9777
Epoch 10/10
703/703 [==============================] - 86s 122m

In [ ]:
# Save the ontology to a file
onto.serialize('/content/medicine_prescription.owl', format='turtle')

<Graph identifier=Nacb463e3548c402cae8bdc8864e26de0 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Convert string labels to integers using LabelEncoder
label_encoder = LabelEncoder()
df['Drug_Name'] = label_encoder.fit_transform(df['Drug_Name'])

# Define the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(input_sequences, df['Drug_Name'], epochs=10)

Epoch 1/10
703/703 [==============================] - 92s 130ms/step - loss: 10.2645 - accuracy: 4.4482e-05
Epoch 2/10
703/703 [==============================] - 88s 125ms/step - loss: 10.0419 - accuracy: 1.7793e-04
Epoch 3/10
703/703 [==============================] - 84s 120ms/step - loss: 9.2675 - accuracy: 0.0020
Epoch 4/10
703/703 [==============================] - 84s 120ms/step - loss: 6.9303 - accuracy: 0.0331
Epoch 5/10
703/703 [==============================] - 84s 120ms/step - loss: 3.7971 - accuracy: 0.2394
Epoch 6/10
703/703 [==============================] - 85s 121ms/step - loss: 1.5940 - accuracy: 0.6065
Epoch 7/10
703/703 [==============================] - 84s 119ms/step - loss: 0.6130 - accuracy: 0.8530
Epoch 8/10
703/703 [==============================] - 84s 120ms/step - loss: 0.2337 - accuracy: 0.9516
Epoch 9/10
703/703 [==============================] - 84s 120ms/step - loss: 0.0908 - accuracy: 0.9847
Epoch 10/10
703/703 [==============================] - 83s 118m

In [ ]:
# Function to predict medicine for a given disease
def predict_medicine(Description):
    # Preprocess the input disease
    input_text = tokenizer.texts_to_sequences([Description])
    padded_input = pad_sequences(input_text, maxlen=max_sequence_len, padding='post')

    # Make prediction
    prediction = model.predict(padded_input)
    predicted_class = np.argmax(prediction)

    # Convert predicted class back to original medicine name using inverse transform
    predicted_medicine = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_medicine

# Example usage
input_disease = "Acne"  # Replace with the actual disease input
predicted_medicine = predict_medicine(input_disease)
print(f"For the disease '{input_disease}', the predicted medicine is: {predicted_medicine}")

1/1 [==============================] - 0s 23ms/step
For the disease 'Acne', the predicted medicine is: 16961
